In [ ]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client['RestaurantsDB']
collection = db['restaurants']

In [ ]:
#Question 1 - Find the number of restaurants in each borough that have a grade input by the user
pipeline = [
    {"$unwind": "$grades"},
    {"$match": {
        "grades.grade": "A",
        "borough": {"$ne": "Missing"}  # Exclude documents with borough "Missing"
    }},
    {"$group": {"_id": "$borough", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}},
    {"$limit": 10}
]

result = collection.aggregate(pipeline)
for entry in result:
    print(entry)

In [ ]:
#Question 2 - Print the location of the top 5 restaurants in the database
pipeline = [
    {"$unwind": "$grades"},
    {"$group": {
        "_id": {
            "name": "$name",
            "location": "$address.coord"
        },
        "averageScore": {"$avg": "$grades.score"}
    }},
    {"$sort": {"averageScore": -1}},
    {"$limit": 5}
]

result = collection.aggregate(pipeline)

for entry in result:
    print(entry)

In [ ]:
# Q3: Count the restaurants that have received at least one 'A' grade and at least one 'B' grade (limit 10)

query = {"grade": {"$in": ['A', 'B', 'C']}}
    
grades_count = collection.count_documents(query)

print(grades_count)


In [ ]:
pipeline = [
    {"$match": {"borough": "Brooklyn"}},
    {"$unwind": "$grades"},
    {"$group": {
        "_id": {"cuisine": "$cuisine"},
        "averageGrade": {"$avg": "$grades.score"},
        "count": {"$sum": 1}
    }},
    {"$match": {"averageGrade": {"$eq": "B"}}}
]

result = collection.aggregate(pipeline)

for entry in result:
    print(entry)


In [ ]:
# Define the find query and projection
find_query = {
    "borough": "Manhattan",
    "cuisine": "Italian",
    "grades.grade": "A"
}

# Create the aggregation pipeline
pipeline = [
    {"$match": find_query},
    {"$project": {
        "_id": 0,
        "name": 1,
        "grades": {
            "$filter": {
                "input": "$grades",
                "as": "grade",
                "cond": {"$eq": ["$$grade.grade", "A"]}
            }
        },
    }},
    {"$limit": 10}
]

# Execute the aggregation pipeline using the aggregate method
result = collection.aggregate(pipeline)

# Print the result
for entry in result:
    print(entry)


In [ ]:
result = collection.aggregate([
    {"$group": {"_id": "$cuisine", "count": {"$sum": 1}}},
    {"$limit": 10}, {"$sort": {"count": -1}}
])

for entry in result:
    print(entry)

In [ ]:

result = collection.aggregate([
    {"$addFields": {
        "firstGradeScore": {"$arrayElemAt": ["$grades.score", -1]},
        "lastGradeScore": {"$arrayElemAt": ["$grades.score", 0]},
    }},
    {"$addFields": {
        "scoreImprovement": {"$subtract": ["$lastGradeScore", "$firstGradeScore"]}
    }},
    {"$match": {"scoreImprovement": {"$gte": 10}}},
    {"$group": {
        "_id": {
            "name": "$name",
            "scoreImprovement": "$scoreImprovement"
        }}},
    {"$limit": 10},
    {"$sort": {"_id.scoreImprovement": -1}}
])

for entry in result:
    print(entry)

In [ ]:
collection.aggregate([
    { $group: { _id: "$cuisine", count: { $sum: 1 } } },
    { $limit: 10 }
])